In [3]:
import os
import pandas as pd
import pyreadstat
from tqdm import tqdm

## Get all datasets properties and store them

### Code

The following code **iterates over all files** in the root directory containing all SHARE datasets. It extracts the properties of each dataset and stores them in a pandas dataframe. The properties are:
- dataset name
- wave
- number of rows
- number of columns
- column names

### Details

- For some reason, one or multiple files, when trying to read them, throw a `ValueError`. However, `pandas` gives us the solution by telling us we should add `convert_categoricals=False` to the `read_csv` function when this is happening. This is the reason why we have a `try` and `except` block in the code.

In [6]:
# initiate constants
directory = 'data/'
file_names = []
waves = []
n_rows = []
n_columns = []
columns = []
labels = []

# special case that we want to skip
def is_special_case(file):
    """
    Check if the file is a special case that we want to skip.
    Special cases are files such as imputation files and technical variables.

    Parameters:
    file (str): The name of the file.

    Returns:
    bool: True if the file is a special case, False otherwise.
    """
    
    special_cases = [
        'imputation',
        'technical_variables',
        'dropoff',
        'children',
        'exrates',
        'vignettes',
        'weights',
        'interviewer',
        'ilextra',
        'cv_r'
    ]
    for special_case in special_cases:
        if special_case in file:
            return True

# iterate through all files in the directory
for root, dirs, files in tqdm(os.walk(directory)):
    for file in files:
        if file.endswith('.dta'):

            # skip special cases
            if is_special_case(file):
                # skip and continue with the next file
                continue 
            try:
                dataset, meta = pyreadstat.read_dta(os.path.join(root, file))

                column_names = list(meta.column_names)
                column_labels = [label.replace(',', '') for label in meta.column_labels]

                if len(column_names) != len(column_labels):
                    print(f'different number of column names and labels in {file}')
                    print(f'column names: {len(column_names)}')
                    print(f'column labels: {len(column_labels)}')
            except:
                print('Error when opening:\n ', os.path.join(root, file))

            # get all meta data from current file
            file_names.append(file)
            waves.append(file[6])
            n_rows.append(len(dataset))
            n_columns.append(len(dataset.columns))
            columns.append(column_names)
            labels.append(column_labels)

# create a dataframe with the results
df = pd.DataFrame({
    'file_name': file_names,
    'wave': waves,
    'n_rows': n_rows,
    'n_columns': n_columns,
    'columns': columns,
    'labels': labels
})

5it [00:47,  8.98s/it]

Error when opening:
  data/sharew7_rel9-0-0_ALL_datasets_stata/sharew7_rel9-0-0_ra.dta


10it [02:36, 15.65s/it]


In [7]:
df.sample(10)

,file_name,wave,n_rows,n_columns,columns,labels
89,sharew2_rel9-0-0_hh.dta,2,37132,21,"[mergeid, hhid2, mergeidp2, coupleid2, country...",[Person identifier (fix across modules and wav...
191,sharew6_rel9-0-0_dn.dta,6,68055,145,"[mergeid, hhid6, mergeidp6, coupleid6, country...",[Person identifier (fix across modules and wav...
210,sharew3_rel9-0-0_dq.dta,3,28454,177,"[mergeid, hhid3, mergeidp3, coupleid3, country...",[Person identifier (fix across modules and wav...
18,sharew5_rel9-0-0_iv.dta,5,66038,27,"[mergeid, hhid5, mergeidp5, coupleid5, country...",[Person identifier (fix across modules and wav...
211,sharew3_rel9-0-0_hs.dta,3,28454,175,"[mergeid, hhid3, mergeidp3, coupleid3, country...",[Person identifier (fix across modules and wav...
96,sharew2_rel9-0-0_br.dta,2,37132,23,"[mergeid, hhid2, mergeidp2, coupleid2, country...",[Person identifier (fix across modules and wav...
149,sharew8_rel9-0-0_gs.dta,8,53695,23,"[mergeid, hhid8, mergeidp8, coupleid8, country...",[Person identifier (fix across modules and wav...
134,sharew7_rel9-0-0_ch.dta,7,77181,1616,"[mergeid, hhid7, mergeidp7, coupleid7, country...",[Person identifier (fix across modules and wav...
37,sharew4_rel9-0-0_xt.dta,4,1174,172,"[mergeid, hhid4, country, language_xt, gender_...",[Person identifier (fix across modules and wav...
2,sharew5_rel9-0-0_ac.dta,5,66038,38,"[mergeid, hhid5, mergeidp5, coupleid5, country...",[Person identifier (fix across modules and wav...


## Discriminate normal and particular datasets

According to the [official documentation](https://share-eric.eu/data/faqs-support):

*"The naming of variables is harmonised across waves. Variable names in the CAPI instrument data use the following format: mmXXXyyy_LL. “mm” is the module identifier, e.g. DN for the demographics module, “XXX” refers to the question number, e.g. 001, and “yyy” are optional digits for dummy variables (indicated by “d”), euro conversion (indicated by “e”) or unfolding brackets (indicated by “ub”). The separation character “_” is followed by “LL” optional digits for category or loop indication (“outer loop”)."*

For this reason, we add **boolean indicator** columns to the dataframe to discriminate normal datasets from particular ones. This allows us to see that approximately 10% of the datasets are particular, and maybe useless for us.

In [28]:
def is_file_normal(file_name: str, only_gv: bool=True) -> bool:
    """
    Detect if the last 3 elements of the string follow the pattern _ab, 
    where 'a' and 'b' are single letters.
    
    Args:
    - file_name (str): The string to be checked.
    Returns:
    - bool: True if the pattern is found, False otherwise.
    """

    # special case the General Variables files
    if '_gv_' in file_name:
        return True
    
    if only_gv:
        return False
    
    # get file extension
    suffix = file_name[:-4]

    # check if the suffix is long enough
    if len(suffix) < 3:
        return False
    
    # check if the last three elements follow the pattern
    last_three = suffix[-3:]
    if last_three[0] == '_' and last_three[1].isalpha() and last_three[2].isalpha():
        return True
    else:
        return False
    
# filter out the files that are not normal
df['is_normal'] = df['file_name'].apply(is_file_normal, only_gv=False)
df['is_gv'] = df['file_name'].apply(is_file_normal, only_gv=True)

# save the dataframe to a csv file
df.to_csv('data_info.csv', index=False)
df.sample(10)

,file_name,wave,n_rows,n_columns,columns,labels,is_normal,is_gv
218,sharew9_rel9-0-0_br.dta,9,69447,25,"[mergeid, hhid9, mergeidp9, coupleid9, country...",[Person identifier (fix across modules and wav...,True,False
210,sharew3_rel9-0-0_dq.dta,3,28454,177,"[mergeid, hhid3, mergeidp3, coupleid3, country...",[Person identifier (fix across modules and wav...,True,False
8,sharew5_rel9-0-0_ex.dta,5,66038,23,"[mergeid, hhid5, mergeidp5, coupleid5, country...",[Person identifier (fix across modules and wav...,True,False
54,sharew1_rel9-0-0_iv.dta,1,30416,42,"[mergeid, hhid1, mergeidp1, coupleid1, country...",[Person identifier (fix across modules and wav...,True,False
28,sharew4_rel9-0-0_ho.dta,4,57982,88,"[mergeid, hhid4, mergeidp4, coupleid4, country...",[Person identifier (fix across modules and wav...,True,False
23,sharew5_rel9-0-0_gv_housing.dta,5,66038,11,"[mergeid, hhid5, mergeidp5, coupleid5, country...",[Person identifier (fix across modules and wav...,True,True
76,sharew2_rel9-0-0_ch.dta,2,37132,223,"[mergeid, hhid2, mergeidp2, coupleid2, country...",[Person identifier (fix across modules and wav...,True,False
47,sharew4_rel9-0-0_sn.dta,4,57982,56,"[mergeid, hhid4, mergeidp4, coupleid4, country...",[Person identifier (fix across modules and wav...,True,False
221,sharew9_rel9-0-0_xt.dta,9,3491,220,"[mergeid, hhid9, country, language_xt, gender_...",[Person identifier (fix across modules and wav...,True,False
213,sharew9_rel9-0-0_iv.dta,9,69447,26,"[mergeid, hhid9, mergeidp9, coupleid9, country...",[Person identifier (fix across modules and wav...,True,False


In [5]:
print(round(df.is_normal.value_counts()/len(df)*100,2), '\n')
print(round(df.is_gv.value_counts()/len(df)*100,2))

is_normal
True    100.0
Name: count, dtype: float64 

is_gv
False    83.26
True     16.74
Name: count, dtype: float64


## Get column names for each wave

We store the column names for each wave in `.csv` files, so we can use them later. The aim is that we don't have to open all files just for the columns names.

For the variable selection feature, we only want to **keep relevant variables**. For this, we filter on variables that are not identifiers (such as `mergeid`).

In [29]:
patterns_to_remove = ['mergeid', 'hhid', 'coupleid'] 

def is_valid_column(col_name, patterns_to_remove=patterns_to_remove):
    """
    Check if a column name is not an identifier.
    
    Example: if a column is named 'mergeid6', it respects the
    pattern 'mergeid' and should be removed.
    
    Args:
    - col_name (str): The column name to be checked.
    - patterns_to_remove (list): A list of patterns to be removed.
    Returns:
    - bool: True if the column name is valid, False otherwise.
    """
    
    for pattern in patterns_to_remove:
        if pattern in col_name:
            return False
    return True

In [35]:
df = pd.read_csv('data_info.csv')

# iterate through the waves
for wave in range(1,9+1):
    columns_properties = pd.DataFrame()
    subset = df[df['wave'] == wave]

    # iterate through the files in the wave
    for columns,labels,filename in zip(subset['columns'],subset['labels'],subset['file_name']):
        
        # get all columns and the file they belong to
        row = list(columns[1:-1].replace("'", "").split(', '))
        label = list(labels[1:-1].replace("'", "").split(', '))
        filenames = [filename]*len(row)

        # add the columns and the file to the dataframe
        print(filenames)
        print(len(filenames))
        temp = pd.DataFrame({
            'column': row,
            'labels': label,
            'file_name': filenames})
        columns_properties = pd.concat([columns_properties, temp])

    columns_properties.drop_duplicates(inplace=True)
    print(f'Wave {wave} has {len(columns)} unique columns.')

    # remove all column names dupplicate (keep first occurence)
    columns_properties.drop_duplicates(subset='column', keep='first', inplace=True)

    # remove all columns that are identifiers
    columns_properties['is_valid'] = columns_properties['column'].apply(is_valid_column)
    columns_properties = columns_properties[columns_properties['is_valid'] == True]

    # save as csv file
    #columns_properties.to_csv(f'columns/wave_{wave}_columns.csv', index=False)

['sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 'sharew1_rel9-0-0_hc.dta', 

ValueError: All arrays must be of the same length

## Create basic dataset per wave

The aim of this code is to create a basic/dummy dataset per wave. This dataset contains the following columns:

- `mergeid`: identifier of the respondent
- `country`: country of the respondent
- `language`: language of the respondent

In [8]:
for wave in tqdm(range(1,9+1)):
    df = pd.read_stata(f'data/sharew{wave}_rel9-0-0_ALL_datasets_stata/sharew{wave}_rel9-0-0_ac.dta')
    df = df[['country', 'mergeid', 'language']]
    df.to_stata(f'data/sharew{wave}_rel9-0-0_ALL_datasets_stata/wave{wave}_dummy.stata', write_index=False)

100%|██████████| 9/9 [00:02<00:00,  4.45it/s]
